
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [35]:
# import libraries
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## Directory

In [36]:
def process_json_files(directory_path, output_csv_path):
    """
    Process JSON files in the specified directory, normalize the data, and save it to a CSV file.

    Parameters:
    - directory_path: str, path to the directory containing JSON files.
    - output_csv_path: str, path to save the output CSV file.

    Returns:
    - None
    """
    # List all JSON files in the specified directory
    files = [f for f in os.listdir(directory_path) if f.endswith('.json')]
    print("Files in directory:")
    print(files)

    # Initialize an empty list to store dataframes
    df_list = []

    # Iterate over each JSON file in the directory
    for file_name in files:
        file_path = os.path.join(directory_path, file_name)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # First level normalization
        df = pd.json_normalize(data['data']['results'])
        
        # Second level normalization
        if 'branding' in df.columns:
            branding = pd.json_normalize(df['branding'].explode())
            branding.columns = [f'branding.{col}' for col in branding.columns]
            df = df.drop('branding', axis=1).join(branding)

        if 'description' in df.columns:
            description = pd.json_normalize(df['description'])
            description.columns = [f'description.{col}' for col in description.columns]
            df = df.drop('description', axis=1).join(description)

        if 'flags' in df.columns:
            flags = pd.json_normalize(df['flags'])
            flags.columns = [f'flags.{col}' for col in flags.columns]
            df = df.drop('flags', axis=1).join(flags)

        if 'lead_attributes' in df.columns:
            lead_attributes = pd.json_normalize(df['lead_attributes'])
            lead_attributes.columns = [f'lead_attributes.{col}' for col in lead_attributes.columns]
            df = df.drop('lead_attributes', axis=1).join(lead_attributes)

        if 'location.address' in df.columns:
            location_address = pd.json_normalize(df['location.address'])
            location_address.columns = [f'location.address.{col}' for col in location_address.columns]
            df = df.drop('location.address', axis=1).join(location_address)

        if 'location.county' in df.columns:
            location_county = pd.json_normalize(df['location.county'])
            location_county.columns = [f'location.county.{col}' for col in location_county.columns]
            df = df.drop('location.county', axis=1).join(location_county)

        if 'products' in df.columns:
            products = pd.json_normalize(df['products'])
            products.columns = [f'products.{col}' for col in products.columns]
            df = df.drop('products', axis=1).join(products)

        # Third level normalization
        if 'location.address.coordinate' in df.columns:
            location_coordinates = pd.json_normalize(df['location.address.coordinate'])
            location_coordinates.columns = [f'location.address.coordinate.{col}' for col in location_coordinates.columns]
            df = df.drop('location.address.coordinate', axis=1).join(location_coordinates)

        if 'source.agents' in df.columns:
            source_agents = pd.json_normalize(df['source.agents'].explode())
            source_agents.columns = [f'source.agents.{col}' for col in source_agents.columns]
            df = df.drop('source.agents', axis=1).join(source_agents)
            
            # Handle duplicate office_name values
            if 'source.agents.office_name' in df.columns:
                df['source.agents.office_name'] = df['source.agents.office_name'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)

        if 'other_listings.rdc' in df.columns:
            other_listings_rdc = pd.json_normalize(df['other_listings.rdc'].explode())
            other_listings_rdc.columns = [f'other_listings.rdc.{col}' for col in other_listings_rdc.columns]
            df = df.drop('other_listings.rdc', axis=1).join(other_listings_rdc)

        # Ensure all-bool object columns are cast to bool dtype
        for col in df.select_dtypes(include=['object']):
            if df[col].dropna().isin([True, False]).all():
                df[col] = df[col].astype(bool)
        
        # Append the dataframe to the list, skipping 'tags' column normalization
        df_list.append(df)

    # Combine all dataframes
    combined_df = pd.concat(df_list, ignore_index=True)

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(output_csv_path, index=False)

    print(f"Data saved to {output_csv_path}")

In [37]:
# example usage
directory_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data'
output_csv_path = 'e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data/processed_data.csv'
process_json_files(directory_path, output_csv_path)

Files in directory:
['AK_Juneau_0.json', 'AK_Juneau_1.json', 'AK_Juneau_2.json', 'AK_Juneau_3.json', 'AK_Juneau_4.json', 'AL_Montgomery_0.json', 'AL_Montgomery_1.json', 'AL_Montgomery_2.json', 'AL_Montgomery_3.json', 'AL_Montgomery_4.json', 'AR_LittleRock_0.json', 'AR_LittleRock_1.json', 'AR_LittleRock_2.json', 'AR_LittleRock_3.json', 'AR_LittleRock_4.json', 'AZ_Phoenix_0.json', 'AZ_Phoenix_1.json', 'AZ_Phoenix_2.json', 'AZ_Phoenix_3.json', 'AZ_Phoenix_4.json', 'CA_Sacramento_0.json', 'CA_Sacramento_1.json', 'CA_Sacramento_2.json', 'CA_Sacramento_3.json', 'CA_Sacramento_4.json', 'CO_Denver_0.json', 'CO_Denver_1.json', 'CO_Denver_2.json', 'CO_Denver_3.json', 'CO_Denver_4.json', 'CT_Hartford_0.json', 'CT_Hartford_1.json', 'CT_Hartford_2.json', 'CT_Hartford_3.json', 'CT_Hartford_4.json', 'DE_Dover_0.json', 'DE_Dover_1.json', 'DE_Dover_2.json', 'DE_Dover_3.json', 'DE_Dover_4.json', 'FL_Tallahassee_0.json', 'FL_Tallahassee_1.json', 'FL_Tallahassee_2.json', 'FL_Tallahassee_3.json', 'FL_Talla

At this point, ensure that you have all sales in a dataframe.

In [38]:
df = pd.read_csv(output_csv_path, low_memory=False)

In [39]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

- Is each cell one value, or do some cells have lists?

In [40]:
df.head()

,last_update_date,tags,permalink,status,list_date,open_houses,list_price,property_id,photos,community,virtual_tours,listing_id,price_reduced_amount,matterport,primary_photo.href,source.plan_id,source.spec_id,source.type,description.year_built,description.baths_3qtr,description.sold_date,description.sold_price,description.baths_full,description.name,description.baths_half,description.lot_sqft,description.sqft,description.baths,description.sub_type,description.baths_1qtr,description.garage,description.stories,description.beds,description.type,lead_attributes.show_contact_an_agent,flags.is_new_construction,flags.is_for_rent,flags.is_subdivision,flags.is_contingent,flags.is_price_reduced,flags.is_pending,flags.is_foreclosure,flags.is_plan,flags.is_coming_soon,flags.is_new_listing,products.brand_name,location.address.postal_code,location.address.state,location.address.coordinate.lon,location.address.coordinate.lat,location.address.city,location.address.state_code,location.address.line,location.street_view_url,location.county.fips_code,location.county.name,primary_photo,source,other_listings,branding.name,branding.photo,branding.type,source.agents.office_name,other_listings.rdc.listing_id,other_listings.rdc.listing_key,other_listings.rdc.status,other_listings.rdc.primary,community.advertisers,community.description.name
0,2023-09-19T20:52:50Z,"['carport', 'community_outdoor_space', 'cul_de...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,False,554950.0,9.074431e+09,"[{'tags': [{'label': 'house_view', 'probabilit...",False,False,2957241843,45000.0,False,https://ap.rdcpix.com/07097d34c98a59ebb7996889...,NaN,NaN,mls,1963.0,False,2023-09-18,False,2.0,False,False,10454.0,1821.0,2.0,NaN,False,1.0,NaN,3.0,single_family,True,False,False,False,False,False,False,False,False,False,False,basic_opt_in,99801.0,Alaska,-134.59372,58.36395,Juneau,AK,9453 Herbert Pl,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,EXP Realty LLC - Southeast Alaska,False,Office,EXP Realty LLC - Southeast Alaska,2.957242e+09,NaN,sold,True,NaN,NaN
1,NaN,NaN,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,NaN,False,NaN,9.424984e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-22,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,8477 Thunder Mountain Rd,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,Non-Member Office,2.957023e+09,NaN,off_market,False,NaN,NaN
2,NaN,NaN,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,NaN,False,NaN,9.479069e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-22,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,4515 Glacier Hwy,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958935e+09,NaN,sold,False,NaN,NaN
3,NaN,NaN,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,NaN,False,NaN,9.879332e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-21,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,17850 Point Stephens Rd,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958935e+09,NaN,sold,False,NaN,NaN
4,NaN,NaN,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,NaN,False,NaN,9.521640e+09,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,2023-08-21,False,NaN,False,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,NaN,99801.0,Alaska,NaN,NaN,Juneau,AK,9951 Stephen Richards Memorial Dr,https://maps.googleapis.com/maps/api/streetvie...,False,Juneau,NaN,NaN,NaN,NaN,False,Office,NaN,2.958925e+

- What are the data types of each column?

In [14]:
# display all columns and their data types
print("Columns and their data types:")
print(df.dtypes)

Columns and their data types:
last_update_date                          object
tags                                      object
permalink                                 object
status                                    object
list_date                                 object
open_houses                               object
list_price                                object
property_id                              float64
photos                                    object
community                                 object
virtual_tours                             object
listing_id                                object
price_reduced_amount                      object
matterport                                object
primary_photo.href                        object
source.plan_id                            object
source.spec_id                            object
source.type                               object
description.year_built                    object
description.baths_3qtr                 

- Some sales may not actually include the sale price. These rows should be dropped.

In [25]:
# number of rows before dropping
rows_before = len(df)

# drop rows without 'description.sold_price' (target variable)
df = df.dropna(subset=['description.sold_price'])

# number of rows after dropping
rows_after = len(df)

# calculate the number of rows dropped
rows_dropped = rows_before - rows_after

# print the number of rows dropped
print(f"Number of rows dropped: {rows_dropped}")

Number of rows dropped: 37


- There are a lot of None values.  Should these be dropped or replaced with something?

In [26]:
# check the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_percentage)

Percentage of missing values in each column:
last_update_date                          0.416973
tags                                      6.217807
permalink                                 0.000000
status                                    0.000000
list_date                                 4.881040
open_houses                               0.000000
list_price                                5.261221
property_id                               0.000000
photos                                    8.891342
community                                 2.575423
virtual_tours                            74.540103
listing_id                                4.881040
price_reduced_amount                     68.297768
matterport                                0.000000
primary_photo.href                        9.271523
source.plan_id                           17.525141
source.spec_id                           17.525141
source.type                               4.991415
description.year_built               

In [27]:
# calculate the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100

# define a threshold for dropping columns (e.g., rule of thumb: more than 50% missing values)
threshold = 50

# identify columns to drop based on the threshold
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# drop the identified columns from the DataFrame
df = df.drop(columns=columns_to_drop)

# print the dropped columns
print(f"Dropped columns: {columns_to_drop}")

# display the remaining columns
print("Remaining columns after dropping:")
print(df.columns)

Dropped columns: Index(['virtual_tours', 'price_reduced_amount', 'description.baths_half',
       'description.sub_type', 'primary_photo', 'source', 'other_listings',
       'branding.photo', 'source.agents.office_name', 'community.advertisers',
       'community.description.name'],
      dtype='object')
Remaining columns after dropping:
Index(['last_update_date', 'tags', 'permalink', 'status', 'list_date',
       'open_houses', 'list_price', 'property_id', 'photos', 'community',
       'listing_id', 'matterport', 'primary_photo.href', 'source.plan_id',
       'source.spec_id', 'source.type', 'description.year_built',
       'description.baths_3qtr', 'description.sold_date',
       'description.sold_price', 'description.baths_full', 'description.name',
       'description.lot_sqft', 'description.sqft', 'description.baths',
       'description.baths_1qtr', 'description.garage', 'description.stories',
       'description.beds', 'description.type',
       'lead_attributes.show_contact_an_

In [ ]:
'list_date'
'description.year_built'
'description.baths_3qtr'
'description.sold_date'
'description.sold_price'
'description.baths_full'
'description.lot_sqft'
'description.sqft'
'description.baths'
'description.garage'
'description.stories'
'description.beds'
'description.type'

In [ ]:
'tags'
'location.address.state'
'location.address.city'
'location.county.name'
'branding.name'
'flags.is_price_reduced'
'flags.is_foreclosure'

In [72]:
# list of irrelevant columns to drop
columns_to_drop = ['last_update_date', 'permalink', 'status', 'open_houses', 'list_price', 'property_id', 'photos', 'community', 
                   'listing_id', 'matterport', 'primary_photo.href', 'source.plan_id', 'source.spec_id', 'source.type', 
                   'location.street_view_url', 'matterport', 'description.name', 'description.baths_1qtr', 
                   'lead_attributes.show_contact_an_agent', 'flags.is_new_construction', 'flags.is_for_rent', 'flags.is_subdivision', 
                   'flags.is_contingent', 'flags.is_pending', 'flags.is_plan', 'flags.is_coming_soon', 'flags.is_new_listing', 
                   'products.brand_name', 'location.address.postal_code', 'location.address.coordinate.lon', 'location.address.coordinate.lat', 
                   'location.address.state_code', 'location.address.line', 'location.street_view_url', 'location.county.fips_code', 'branding.type', 
                   'other_listings.rdc.listing_id', 'other_listings.rdc.listing_key', 'other_listings.rdc.status', 'other_listings.rdc.primary']

# drop the specified columns
df = df.drop(columns=columns_to_drop)

- Some sales don't include the property type.

In [73]:
# impute missing values for categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer.fit_transform(df[categorical_cols])

In [74]:
# impute missing values for numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns
imputer = SimpleImputer(strategy='median')
df[numerical_cols] = imputer.fit_transform(df[numerical_cols])

In [42]:
# check the percentage of missing values in each column
missing_percentage = df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_percentage)

Percentage of missing values in each column:
tags                                     0.0
list_date                                0.0
property_id                              0.0
matterport                               0.0
description.year_built                   0.0
description.baths_3qtr                   0.0
description.sold_date                    0.0
description.sold_price                   0.0
description.baths_full                   0.0
description.name                         0.0
description.lot_sqft                     0.0
description.sqft                         0.0
description.baths                        0.0
description.baths_1qtr                   0.0
description.garage                       0.0
description.stories                      0.0
description.beds                         0.0
description.type                         0.0
lead_attributes.show_contact_an_agent    0.0
flags.is_new_construction                0.0
flags.is_for_rent                        0.0
flags.is_s

In [ ]:
# Feature engineer new column --> days on the market

# sold_date - list_date

In [43]:
# function to cap outliers at a specified percentile
def cap_outliers(df, cols, lower_percentile=0.05, upper_percentile=0.95):
    for col in cols:
        lower_bound = df[col].quantile(lower_percentile)
        upper_bound = df[col].quantile(upper_percentile)
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

# identify all numerical columns to cap outliers
numerical_cols_to_cap = df.select_dtypes(include=['number']).columns

# cap outliers in these numerical columns
df = cap_outliers(df, numerical_cols_to_cap)

- Maybe the "tags" will help create some features.

Consider the fact that with tags, there are a lot of categorical variables.


- How many columns would we have if we OHE tags, city and state?

- Perhaps we can get rid of tags that have a low frequency.

In [17]:
def one_hot_encode_and_filter(df, columns, low_freq_threshold=5):
    """
    One-hot encode specified columns and drop columns with low frequency.

    Parameters:
    - df: pandas DataFrame, the dataframe to process
    - columns: list of str, columns to one-hot encode
    - low_freq_threshold: int, frequency threshold to drop low frequency columns

    Returns:
    - pandas DataFrame, the processed dataframe
    """
    for col in columns:
        dummies = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(col, axis=1)

    # Calculate the frequency of each one-hot encoded column
    frequency = df.sum(numeric_only=True)

    # Identify columns with frequency below the threshold
    low_freq_columns = frequency[frequency < low_freq_threshold].index

    # Drop columns with low frequency
    df = df.drop(low_freq_columns, axis=1)
    
    return df

In [ ]:
'tags'
'location.address.state'
'location.address.city'
'location.county.name'
'branding.name'
'flags.is_price_reduced'
'flags.is_foreclosure'

In [75]:
# Count the number of unique values in the 'tags' column
num_unique_values = df['tags'].nunique()
print(f"Number of unique values in 'tags': {num_unique_values}")

Number of unique values in 'tags': 1629


In [18]:
# example usage
df = df
columns = ['tags', 'location.address.city', 'location.address.state']
one_hot_encode_and_filter(df, columns, low_freq_threshold=5)

# One-hot encode 'tags', 'location.address.city', and 'location.address.state' and filter low frequency columns
df = one_hot_encode_and_filter(df, ['tags', 'location.address.city', 'location.address.state'])

# Save the combined DataFrame to a CSV file
df.to_csv(output_csv_path, index=False)
print(f"Data saved to {output_csv_path}")


Data saved to e:/Vocational/Lighthouse Labs/Flex Course/Projects/P02_Midterm_Supervised Learning/data_project_midterm/data/processed_data.csv


- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [19]:
# split the DataFrame into training (70%) and test (30%) sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [20]:
# save the split data to CSV files if needed
train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# Load Data
df = pd.read_csv('path_to_your_data/processed_data.csv')

In [ ]:
# 1. Summary Statistics
def summary_statistics(df):
    print("Summary Statistics:")
    print(df.describe())
    print("\nData Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())

summary_statistics(df)

In [ ]:
# 2. Check for Missing Values
def missing_values(df):
    print("\nMissing Values:")
    print(df.isnull().sum())

missing_values(df)

In [ ]:
# 3. Distributions of Numerical Variables
def plot_distributions(df):
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        plt.figure(figsize=(10, 6))
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribution of {col}')
        plt.show()

plot_distributions(df)

In [ ]:
# 4. Scatterplots for Numerical Variables and Target
def scatterplots(df, target_col):
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    numerical_cols = [col for col in numerical_cols if col != target_col]
    for col in numerical_cols:
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=df[col], y=df[target_col])
        plt.title(f'Scatterplot of {col} vs {target_col}')
        plt.show()

target_col = 'sold_price'
scatterplots(df, target_col)

In [ ]:
# 5. Correlation Heatmap
def correlation_heatmap(df):
    plt.figure(figsize=(12, 10))
    corr_matrix = df.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()

correlation_heatmap(df)

In [ ]:
# 6. Redundant Features
def check_redundant_features(df):
    corr_matrix = df.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    redundant_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.8)]
    print("Redundant Features:")
    print(redundant_features)

check_redundant_features(df)

In [ ]:
# 7. Boxplots to Identify Outliers
def plot_boxplots(df):
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    for col in numerical_cols:
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')
        plt.show()

plot_boxplots(df)

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized